## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-07-18-50-35 +0000,cbc,Gold prices hit record high as anxious investo...,https://www.cbc.ca/news/business/gold-prices-r...
1,2025-10-07-18-44-56 +0000,wapo,Trump administration intervenes to secure woma...,https://www.washingtonpost.com/national-securi...
2,2025-10-07-18-43-07 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/07/us/tru...
3,2025-10-07-18-42-29 +0000,nyt,The Shutdown Dispute Over Health Care Subsidie...,https://www.nytimes.com/2025/10/07/us/politics...
4,2025-10-07-18-33-13 +0000,nyt,Witkoff and Kushner Expected to Join Israel-Ha...,https://www.nytimes.com/2025/10/07/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2396/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,51
20,shutdown,18
17,gaza,16
227,new,16
30,israel,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
246,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...,127
15,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...,119
199,2025-10-07-08-00-00 +0000,cbc,"'Canada will love us again,' Trump says as he ...",https://www.cbc.ca/news/politics/trump-carney-...,113
229,2025-10-07-01-27-00 +0000,wsj,President Trump signaled a willingness to stri...,https://www.wsj.com/politics/policy/trump-open...,110
280,2025-10-06-21-45-09 +0000,latimes,Trump advisors amp up extreme rhetoric against...,https://www.latimes.com/politics/story/2025-10...,107


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
246,127,2025-10-06-22-57-48 +0000,nypost,Senate Democrats reject latest effort to end g...,https://nypost.com/2025/10/06/us-news/senate-d...
15,65,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...
253,63,2025-10-06-22-37-00 +0000,wsj,The Reserve Bank of New Zealand has establishe...,https://www.wsj.com/articles/new-zealand-centr...
49,62,2025-10-07-17-01-48 +0000,latimes,Supreme Court sees a free-speech problem with ...,https://www.latimes.com/politics/story/2025-10...
79,50,2025-10-07-15-50-55 +0000,nypost,NY woman refuses to pay $440 fine to Hertz aft...,https://nypost.com/2025/10/07/business/ny-woma...
60,49,2025-10-07-16-46-07 +0000,nyt,Israel Marks Two-Year Anniversary of Oct. 7 in...,https://www.nytimes.com/2025/10/07/world/middl...
318,40,2025-10-06-19-34-39 +0000,cbc,Bari Weiss named editor-in-chief of CBS News u...,https://www.cbc.ca/news/entertainment/bari-wei...
292,36,2025-10-06-21-05-53 +0000,nypost,Virginia AG candidate Jay Jones said if more c...,https://nypost.com/2025/10/06/us-news/virginia...
161,34,2025-10-07-10-00-00 +0000,nypost,Hostage Eli Sharabi was starved and beaten by ...,https://nypost.com/2025/10/07/world-news/eli-s...
78,32,2025-10-07-15-56-00 +0000,bbc,Time may be running out for Master of clocks M...,https://www.bbc.com/news/articles/c0kn04453z8o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
